In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Hyper params...
num_epochs = 5
batch_size = 16
learning_rate = 0.001

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5, 0.5, 0.5), 
            (0.5, 0.5, 0.5)
        )
    ]
)

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root= '/data',
    train = True, 
    download = True, 
    transform = transform
)

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(
    root= '/data',
    train = False, 
    download = True, 
    transform = transform
)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False 
)
print(len(train_loader))
print(len(test_loader))

In [ ]:
classes = [
    'plane', 'car', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
    ]

In [ ]:
def imshow(img):
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1,2,0)))
  plt.show()

dataiter = iter(train_loader)
images, lables = next(dataiter)

imshow(torchvision.utils.make_grid(images))

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()

    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16, 5)
    # self.pool = nn.MaxPool(2,2)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)


  def forward(self, x):
    x = self.conv1(x) # 28x28x6
    x = F.relu(x)
    x = self.pool(x)  # 14x14x6
    x = self.conv2(x) # 10x10x16
    x = F.relu(x)
    x = self.pool(x)  # 5x5x16

    x = x.view(-1, 16*5*5)  # n, 400
    x = F.relu(self.fc1(x))         # 120
    x = F.relu(self.fc2(x))         # 84
    x = F.relu(self.fc3(x))         # 10

    return x

In [ ]:
model = ConvNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate
)

In [ ]:
n_total_steps = len(train_loader)

In [ ]:
### Training process
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):

    images = images.to(device)
    labels = labels.to(device)
    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1)% 500 ==0:
      print(f'Epoch {epoch+1}/ {num_epochs}, Step {i+1}/{n_total_steps}, Loss = {loss.item():.4f}')

In [ ]:
# Training is done, time to evaluate the model:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device) 
        
        outputs = model(images).to(device)
        # print(outputs.shape)
        # print(outputs[0])

        # max returns value, index
        _, predicted = torch.max(outputs, 1)
        # predicted = predicted.to(device)

        n_samples += lables.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            if (label == pred):
                n_class_correct[label] +=1
            n_class_samples[label] +=1

        acc = 100 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')